<a href="https://colab.research.google.com/github/sanAkel/ocean-hurricane/blob/main/plot_aviso_fields_hurr_track.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Plot AVISO:
- Geostrophic currents
- ADT
- SLA

# Overlay storm track

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import xarray as xr
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
!pip install cartopy

In [ ]:
import cartopy.crs as ccrs

In [ ]:
# mount drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
def plot_aviso_var_track(ds_track, ds_aviso, year, hurr_name, vName="U", DPI=120):
  nSubplots = len(ds_aviso.time)
  #print(nSubplots)
  nRows = int(np.ceil(nSubplots/4))

  if vName == "U":
    vMin, vMax, cMap = [0., 1., "Blues"]
    track_color = 'r'
  elif vName == "adt":
    vMin, vMax, cMap = [-1., 1., "gist_ncar"]
    track_color = 'k'
  elif vName == "sla":
    vMin, vMax, cMap = [-1., 1., "bwr"]
    track_color = 'k'

  fig=plt.figure(figsize=(12, 10))
  for iPlot in range(0, nSubplots):

    date1 = ds_aviso.time[iPlot].values
    title_str=date1.astype(str).split('T')[0]
    #print(title_str)

    ax=plt.subplot(nRows, 4, iPlot+1, projection=ccrs.PlateCarree())
    ax.coastlines()

    ds_aviso[vName].isel(time=iPlot).sel(latitude=slice(ds_track.lat.min().values-10, ds_track.lat.max().values+10),\
                         longitude=slice(ds_track.lon.min().values-10, ds_track.lon.max().values+10)).\
    plot(ax=ax, vmin=vMin, vmax=vMax, cmap=cMap, add_colorbar=False, transform=ccrs.PlateCarree())
    ax.set_title(title_str)

    if date1 >= ds_track.time[0].values and date1 <= ds_track.time[-1].values:
      #print(date1)
      #ax.plot(ds_track.lon, ds_track.lat, ls='--', c='r', transform=ccrs.PlateCarree())
      ax.scatter(ds_track.sel(time=slice(ds_track.time[0], date1)).lon,\
            ds_track.sel(time=slice(ds_track.time[0], date1)).lat,\
            s=8, marker='+', c=track_color, transform=ccrs.PlateCarree())

  plt.subplots_adjust(wspace=0.01, hspace=-0.4)
  figName = str(year) + '_' + hurr_name + '_aviso_' + vName + '.png'
  plt.savefig(figName, dpi=DPI)
  plt.close('all')

## User inputs

In [ ]:
year=2024
myBasin='north_atlantic'
cat_threshold = 4

data_path = '/content/drive/MyDrive/datasets/hurr/'

## Plot all (major) storms of chosen year

In [ ]:
track_summ_fName = data_path + '{}/'.format(year) + 'hurdat2_{}_{}.csv'.format(myBasin, year)
#print("Reading {} summary data from:\n{}".format(year, track_summ_fName))

season_data=pd.read_csv(track_summ_fName)
major_hurr_names = season_data['name'][season_data['category'] >=cat_threshold]

for hurr_name in major_hurr_names:
  print(hurr_name)

  track_fName = "{}/{}/{}_{}_{}.nc".format(data_path, str(year), str(year), myBasin, hurr_name)
  aviso_fName = "{}/{}/{}_{}_{}_AVISO_L4.nc".format(data_path, str(year), str(year), myBasin, hurr_name)
  #print("{}\n{}".format(track_fName, aviso_fName))
  ds_track = xr.open_dataset(track_fName)
  ds_aviso = xr.open_dataset(aviso_fName)

  U = np.sqrt(ds_aviso.ugos**2 + ds_aviso.vgos**2)
  ds_aviso['U'] = U

  for vName in ['U', 'adt', 'sla']:
    plot_aviso_var_track(ds_track, ds_aviso, year, hurr_name, vName)

In [ ]:
fig_dir = '/content/drive/MyDrive/datasets/hurr/' + 'aviso_fields_storm_track_{}'.format(year)

!mkdir -p $fig_dir
!mv *.png $fig_dir